In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import efel
import json
from pathlib import Path
from natsort import natsorted
%matplotlib notebook

In [ ]:
data_folder = Path("/Users/abuccino/Documents/Codes/modeling/multimodal-fitting/multimodalfitting/data_211028/models/hay_ais_ecode_probe_planar")

In [ ]:
responses = {}
for folder in data_folder.iterdir():
    if 'efeature' not in folder.name and "extracellular" not in folder.name and folder.is_dir():
        responses[folder.name] = []
        
        files = [p for p in folder.iterdir()]
        sweep_ids = [int(p.stem.split("-")[1]) for p in files]
        
        print(sorted(sweep_ids))
        for sweep in natsorted(sweep_ids):
            for file in folder.iterdir():
                if f"-{sweep}." in file.name:
                    print(file.name)
                    data = pd.read_csv(file)
                    responses[folder.name].append(data)


In [ ]:
response = responses["IDrest"]
for i, resp in enumerate(response):
    plt.figure()
    plt.plot(resp["time"], resp["voltage"], label=f"Sweep{i}")
    plt.title(f"Sweep{i}")

In [ ]:
idrest_150 = response[2]

In [ ]:
plt.figure()
plt.plot(idrest_150["time"], idrest_150["voltage"])

In [ ]:
trace = {}
trace["T"] = idrest_150["time"]
trace["V"] = idrest_150["voltage"]
trace["stim_start"] = [250]
trace["stim_end"] = [250 + 1350]

efel.setDoubleSetting("interp_step", 0.1)

In [ ]:
feats = efel.getFeatureValues([trace], ["voltage_base", "AHP_depth", "min_AHP_values", "AHP_time_from_peak",
                                        "peak_time", "AP_amplitude", "AP_begin_indices", "AP_duration_half_width",
                                        "AP_fall_indices", "AP_rise_indices"])[0]

In [ ]:
for b in feats["AP_begin_indices"]:
    plt.axvline(b * 0.1, color="y")   
for b in feats["AP_rise_indices"]:
    plt.axvline(b * 0.1, color="g") 
for b in feats["AP_fall_indices"]:
    plt.axvline(b * 0.1, color="r") 

In [ ]:
feats["AP_duration_half_width"]

In [ ]:
(feats["AP_fall_indices"] - feats["AP_rise_indices"]) * 0.1

In [ ]:
for (p, ah) in zip(feats["peak_time"], feats["AHP_time_from_peak"]):
    plt.axvline(p + ah, color="g")
    plt.axvline(p, color="y")    

In [ ]:
feats["min_AHP_values"]

In [ ]:
plt.axhline(feats["voltage_base"], color="r")

In [ ]:
efel.getFeatureNames()